In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd;
import matplotlib.pyplot as plt;
import seaborn as sns;
from sklearn.impute import SimpleImputer;
from sklearn.compose import ColumnTransformer;
from sklearn.pipeline import Pipeline;
from sklearn.preprocessing import LabelEncoder;
from sklearn.preprocessing import StandardScaler;
from sklearn.preprocessing import MinMaxScaler;
from sklearn.model_selection import train_test_split;
from sklearn.linear_model import LinearRegression ;
from sklearn.linear_model import LogisticRegression;
from sklearn.linear_model import Ridge, Lasso;
from sklearn.metrics import mean_squared_error;
from sklearn.metrics import r2_score;
from sklearn.preprocessing import PolynomialFeatures;
from sklearn.svm import SVR;
from sklearn.svm import SVC;
from sklearn.tree import DecisionTreeClassifier;
from sklearn.tree import DecisionTreeRegressor;
from sklearn.ensemble import RandomForestClassifier;
from sklearn.ensemble import RandomForestRegressor;
from sklearn.neighbors import KNeighborsClassifier;
from sklearn.neighbors import KNeighborsRegressor;
from sklearn.naive_bayes import GaussianNB;
import xgboost as xgb;
from xgboost import XGBClassifier;
from xgboost import XGBRegressor;

import tensorflow as tf
import keras;
from keras_preprocessing import image;
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam;
from keras.callbacks import ModelCheckpoint;
from keras.models import Sequential;
from tensorflow.keras.applications import VGG16;
from tensorflow.keras.applications import InceptionResNetV2;
from keras.applications.vgg16 import preprocess_input;
from tensorflow.keras.applications.vgg16 import decode_predictions;
from tensorflow.keras.callbacks import EarlyStopping;

import os;
from os import listdir;
from PIL import Image as PImage;
import cv2

In [2]:
from tqdm import tqdm

We have imported the libraries. 

Its Time to export the data

# Data

In [3]:
train = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/train.csv');
test = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/test.csv')

# How many types of breeds are there?

In [4]:
train['breed'].unique()

array(['nerodia-erythrogaster', 'pantherophis-vulpinus',
       'thamnophis-sirtalis', 'pantherophis-obsoletus',
       'agkistrodon-contortrix', 'crotalus-atrox',
       'lampropeltis-triangulum', 'crotalus-horridus', 'crotalus-ruber',
       'heterodon-platirhinos', 'nerodia-sipedon', 'thamnophis-elegans',
       'thamnophis-marcianus', 'crotalus-viridis', 'nerodia-fasciata',
       'haldea-striatula', 'storeria-dekayi', 'agkistrodon-piscivorus',
       'nerodia-rhombifer', 'storeria-occipitomaculata',
       'thamnophis-radix', 'coluber-constrictor', 'natrix-natrix',
       'diadophis-punctatus', 'masticophis-flagellum',
       'pantherophis-spiloides', 'rhinocheilus-lecontei',
       'lampropeltis-californiae', 'pituophis-catenifer',
       'opheodrys-aestivus', 'pantherophis-guttatus',
       'pantherophis-alleghaniensis', 'thamnophis-proximus',
       'pantherophis-emoryi', 'crotalus-scutulatus'], dtype=object)

In [6]:
train.head()

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix


# X

We have images in train folder. And, we have name of those breeds in the csv format. We have to merge between these two to make the machine understand which image belongs to which breed.

Lets run a simple for loop

In [7]:
img_width = 150;
img_height = 150;
x=[];

for i in tqdm (range(train.shape[0])):
    path = '../input/identify-snake-breed-hackerearth/dataset/train/' + train['image_id'][i] + '.jpg'
    img = image.load_img(path, target_size = (img_width, img_height, 3))
    img = image.img_to_array(img)
    img = img/255.0
    x.append(img)
    
#x = np.array(x);
#rm path

100%|██████████| 5508/5508 [00:16<00:00, 338.98it/s]


Nice, the images are extracted. We have got our x. Now we need y

In [8]:
 y = train.drop(columns = ['image_id'], axis = 1)

In [9]:
y = pd.get_dummies(y)

In [10]:
y = y.to_numpy()

In [ ]:
y

Nice. So, we have converted our y file into numpy array. But how about x? Shouldn't it be a numpy array also?

In [11]:
x = np.array(x)

# Simple Train Test Split

In [12]:
mtrain, mtest, ntrain, ntest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [12]:
mtrain.shape, ntrain.shape

((4406, 150, 150, 3), (4406, 35))

# Deep Model

In [13]:
mod = keras.models.Sequential([
                        keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = mtrain[0].shape),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.30),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.30),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.40),
    
                        keras.layers.Conv2D(16, (3,3), activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.MaxPooling2D(pool_size=(2,2)),
                        keras.layers.Dropout(0.50),
    
                        keras.layers.Flatten(),
     
                        keras.layers.Dense(units = 128, activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dropout(0.50),
    
                        keras.layers.Dense(units = 128, activation = 'relu'),
                        keras.layers.BatchNormalization(),
                        keras.layers.Dropout(0.50),
    
    
                        keras.layers.Dense(units = 35, activation = 'softmax')
])
                        

In [14]:
mod.compile(optimizer= 'Adam' , loss = 'categorical_crossentropy' , metrics= ['accuracy'])

# Training

In [15]:
mod.fit(mtrain, ntrain, epochs= 500, batch_size = 64, validation_data = (mtest, ntest))

Epoch 1/500
69/69 [==============================] - 3s 38ms/step - loss: 4.5876 - accuracy: 0.0350 - val_loss: 3.5919 - val_accuracy: 0.0617
Epoch 2/500
69/69 [==============================] - 2s 28ms/step - loss: 4.2645 - accuracy: 0.0433 - val_loss: 3.7274 - val_accuracy: 0.0617
Epoch 3/500
69/69 [==============================] - 2s 31ms/step - loss: 3.9987 - accuracy: 0.0520 - val_loss: 3.8151 - val_accuracy: 0.0617
Epoch 4/500
69/69 [==============================] - 2s 34ms/step - loss: 3.7955 - accuracy: 0.0710 - val_loss: 3.7804 - val_accuracy: 0.0581
Epoch 5/500
69/69 [==============================] - 2s 34ms/step - loss: 3.6884 - accuracy: 0.0706 - val_loss: 3.7813 - val_accuracy: 0.0617
Epoch 6/500
69/69 [==============================] - 2s 32ms/step - loss: 3.5896 - accuracy: 0.0887 - val_loss: 3.8459 - val_accuracy: 0.0608
Epoch 7/500
69/69 [==============================] - 2s 29ms/step - loss: 3.5446 - accuracy: 0.0933 - val_loss: 4.0219 - val_accuracy: 0.0608
Epoch 

69/69 [==============================] - 2s 27ms/step - loss: 3.0534 - accuracy: 0.1725 - val_loss: 3.4143 - val_accuracy: 0.1053
Epoch 59/500
69/69 [==============================] - 2s 27ms/step - loss: 3.0664 - accuracy: 0.1609 - val_loss: 3.5023 - val_accuracy: 0.0880
Epoch 60/500
69/69 [==============================] - 2s 30ms/step - loss: 3.0494 - accuracy: 0.1668 - val_loss: 3.5152 - val_accuracy: 0.0844
Epoch 61/500
69/69 [==============================] - 2s 28ms/step - loss: 3.0598 - accuracy: 0.1659 - val_loss: 3.3591 - val_accuracy: 0.1034
Epoch 62/500
69/69 [==============================] - 2s 27ms/step - loss: 3.0327 - accuracy: 0.1664 - val_loss: 3.3165 - val_accuracy: 0.1152
Epoch 63/500
69/69 [==============================] - 2s 31ms/step - loss: 3.0362 - accuracy: 0.1650 - val_loss: 3.5140 - val_accuracy: 0.0844
Epoch 64/500
69/69 [==============================] - 2s 28ms/step - loss: 3.0411 - accuracy: 0.1586 - val_loss: 3.3610 - val_accuracy: 0.1116
Epoch 65/500

69/69 [==============================] - 2s 29ms/step - loss: 2.8265 - accuracy: 0.2088 - val_loss: 3.4573 - val_accuracy: 0.1089
Epoch 116/500
69/69 [==============================] - 2s 27ms/step - loss: 2.8231 - accuracy: 0.2129 - val_loss: 3.4337 - val_accuracy: 0.1152
Epoch 117/500
69/69 [==============================] - 2s 28ms/step - loss: 2.8386 - accuracy: 0.2074 - val_loss: 3.3783 - val_accuracy: 0.1125
Epoch 118/500
69/69 [==============================] - 2s 27ms/step - loss: 2.8225 - accuracy: 0.2129 - val_loss: 3.3784 - val_accuracy: 0.1143
Epoch 119/500
69/69 [==============================] - 2s 31ms/step - loss: 2.8273 - accuracy: 0.2077 - val_loss: 3.2843 - val_accuracy: 0.1298
Epoch 120/500
69/69 [==============================] - 2s 30ms/step - loss: 2.8243 - accuracy: 0.2077 - val_loss: 3.3446 - val_accuracy: 0.1261
Epoch 121/500
69/69 [==============================] - 2s 28ms/step - loss: 2.8330 - accuracy: 0.2106 - val_loss: 3.4383 - val_accuracy: 0.1171
Epoch 

Epoch 172/500
69/69 [==============================] - 2s 27ms/step - loss: 2.7096 - accuracy: 0.2379 - val_loss: 3.4592 - val_accuracy: 0.1143
Epoch 173/500
69/69 [==============================] - 2s 27ms/step - loss: 2.7057 - accuracy: 0.2231 - val_loss: 3.4742 - val_accuracy: 0.1198
Epoch 174/500
69/69 [==============================] - 2s 28ms/step - loss: 2.6866 - accuracy: 0.2429 - val_loss: 3.4371 - val_accuracy: 0.1171
Epoch 175/500
69/69 [==============================] - 2s 31ms/step - loss: 2.7070 - accuracy: 0.2306 - val_loss: 3.4303 - val_accuracy: 0.1171
Epoch 176/500
69/69 [==============================] - 2s 28ms/step - loss: 2.7053 - accuracy: 0.2433 - val_loss: 3.4384 - val_accuracy: 0.1252
Epoch 177/500
69/69 [==============================] - 2s 27ms/step - loss: 2.6947 - accuracy: 0.2347 - val_loss: 3.4714 - val_accuracy: 0.1216
Epoch 178/500
69/69 [==============================] - 2s 28ms/step - loss: 2.7076 - accuracy: 0.2419 - val_loss: 3.4173 - val_accuracy:

Epoch 229/500
69/69 [==============================] - 2s 30ms/step - loss: 2.6182 - accuracy: 0.2551 - val_loss: 3.4272 - val_accuracy: 0.1270
Epoch 230/500
69/69 [==============================] - 2s 33ms/step - loss: 2.6101 - accuracy: 0.2581 - val_loss: 3.3594 - val_accuracy: 0.1443
Epoch 231/500
69/69 [==============================] - 2s 27ms/step - loss: 2.6297 - accuracy: 0.2458 - val_loss: 3.4615 - val_accuracy: 0.1234
Epoch 232/500
69/69 [==============================] - 2s 27ms/step - loss: 2.6143 - accuracy: 0.2617 - val_loss: 3.3681 - val_accuracy: 0.1243
Epoch 233/500
69/69 [==============================] - 2s 27ms/step - loss: 2.6287 - accuracy: 0.2551 - val_loss: 3.4513 - val_accuracy: 0.1298
Epoch 234/500
69/69 [==============================] - 2s 28ms/step - loss: 2.6241 - accuracy: 0.2535 - val_loss: 3.4220 - val_accuracy: 0.1198
Epoch 235/500
69/69 [==============================] - 2s 30ms/step - loss: 2.6297 - accuracy: 0.2615 - val_loss: 3.4328 - val_accuracy:

Epoch 286/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5501 - accuracy: 0.2710 - val_loss: 3.4615 - val_accuracy: 0.1307
Epoch 287/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5687 - accuracy: 0.2735 - val_loss: 3.4273 - val_accuracy: 0.1316
Epoch 288/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5491 - accuracy: 0.2667 - val_loss: 3.3583 - val_accuracy: 0.1461
Epoch 289/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5586 - accuracy: 0.2749 - val_loss: 3.4572 - val_accuracy: 0.1252
Epoch 290/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5551 - accuracy: 0.2671 - val_loss: 3.4449 - val_accuracy: 0.1234
Epoch 291/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5667 - accuracy: 0.2653 - val_loss: 3.4105 - val_accuracy: 0.1307
Epoch 292/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5676 - accuracy: 0.2746 - val_loss: 3.3822 - val_accuracy:

Epoch 343/500
69/69 [==============================] - 2s 32ms/step - loss: 2.5136 - accuracy: 0.2860 - val_loss: 3.4423 - val_accuracy: 0.1316
Epoch 344/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5083 - accuracy: 0.2789 - val_loss: 3.4542 - val_accuracy: 0.1316
Epoch 345/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5198 - accuracy: 0.2760 - val_loss: 3.4333 - val_accuracy: 0.1261
Epoch 346/500
69/69 [==============================] - 2s 31ms/step - loss: 2.5055 - accuracy: 0.2846 - val_loss: 3.5536 - val_accuracy: 0.1189
Epoch 347/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5106 - accuracy: 0.2753 - val_loss: 3.4809 - val_accuracy: 0.1289
Epoch 348/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5304 - accuracy: 0.2751 - val_loss: 3.3956 - val_accuracy: 0.1443
Epoch 349/500
69/69 [==============================] - 2s 27ms/step - loss: 2.5361 - accuracy: 0.2724 - val_loss: 3.4335 - val_accuracy:

Epoch 400/500
69/69 [==============================] - 2s 27ms/step - loss: 2.4760 - accuracy: 0.2916 - val_loss: 3.4811 - val_accuracy: 0.1270
Epoch 401/500
69/69 [==============================] - 2s 30ms/step - loss: 2.4808 - accuracy: 0.2848 - val_loss: 3.4849 - val_accuracy: 0.1279
Epoch 402/500
69/69 [==============================] - 2s 28ms/step - loss: 2.4729 - accuracy: 0.2919 - val_loss: 3.5137 - val_accuracy: 0.1316
Epoch 403/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5007 - accuracy: 0.2862 - val_loss: 3.4005 - val_accuracy: 0.1298
Epoch 404/500
69/69 [==============================] - 2s 27ms/step - loss: 2.4856 - accuracy: 0.2941 - val_loss: 3.3973 - val_accuracy: 0.1270
Epoch 405/500
69/69 [==============================] - 2s 28ms/step - loss: 2.5036 - accuracy: 0.2828 - val_loss: 3.4130 - val_accuracy: 0.1252
Epoch 406/500
69/69 [==============================] - 2s 28ms/step - loss: 2.4632 - accuracy: 0.2907 - val_loss: 3.4645 - val_accuracy:

Epoch 457/500
69/69 [==============================] - 3s 40ms/step - loss: 2.4481 - accuracy: 0.2978 - val_loss: 3.4564 - val_accuracy: 0.1352
Epoch 458/500
69/69 [==============================] - 2s 28ms/step - loss: 2.4675 - accuracy: 0.2998 - val_loss: 3.4390 - val_accuracy: 0.1334
Epoch 459/500
69/69 [==============================] - 2s 29ms/step - loss: 2.4621 - accuracy: 0.2951 - val_loss: 3.4167 - val_accuracy: 0.1379
Epoch 460/500
69/69 [==============================] - 2s 28ms/step - loss: 2.4653 - accuracy: 0.2930 - val_loss: 3.4777 - val_accuracy: 0.1261
Epoch 461/500
69/69 [==============================] - 2s 28ms/step - loss: 2.4638 - accuracy: 0.3046 - val_loss: 3.4100 - val_accuracy: 0.1434
Epoch 462/500
69/69 [==============================] - 2s 28ms/step - loss: 2.4553 - accuracy: 0.3064 - val_loss: 3.4731 - val_accuracy: 0.1261
Epoch 463/500
69/69 [==============================] - 2s 27ms/step - loss: 2.4402 - accuracy: 0.2946 - val_loss: 3.4364 - val_accuracy:

# Test Data

In [17]:
img_width = 150;
img_height = 150;
z=[];

for i in tqdm (range(test.shape[0])):
    pathe = '../input/identify-snake-breed-hackerearth/dataset/test/' + test['image_id'][i] + '.jpg'
    img1 = image.load_img(pathe, target_size = (img_width, img_height, 3))
    img1 = image.img_to_array(img1)
    img1 = img1/255.0
    #img1 = img1.reshape(1,img_width, img_height, 3 )
    z.append(img1)

100%|██████████| 2361/2361 [00:06<00:00, 349.89it/s]


In [18]:
z

[array([[[0.3764706 , 0.5529412 , 0.2901961 ],
         [0.35686275, 0.5294118 , 0.2901961 ],
         [0.31764707, 0.49019608, 0.25882354],
         ...,
         [0.3764706 , 0.49019608, 0.40392157],
         [0.3137255 , 0.49019608, 0.32156864],
         [0.26666668, 0.49803922, 0.24705882]],
 
        [[0.38431373, 0.5529412 , 0.28235295],
         [0.5764706 , 0.7411765 , 0.48235294],
         [0.5764706 , 0.7372549 , 0.47058824],
         ...,
         [0.42745098, 0.54509807, 0.44313726],
         [0.6392157 , 0.8       , 0.6666667 ],
         [0.5686275 , 0.7764706 , 0.59607846]],
 
        [[0.26666668, 0.4392157 , 0.12941177],
         [0.24313726, 0.40784314, 0.10980392],
         [0.48235294, 0.63529414, 0.3254902 ],
         ...,
         [0.5686275 , 0.6901961 , 0.5568628 ],
         [0.72156864, 0.8745098 , 0.7607843 ],
         [0.63529414, 0.81960785, 0.69411767]],
 
        ...,
 
        [[0.11372549, 0.17254902, 0.08235294],
         [0.12941177, 0.1882353 , 0.09803

Let's convert it into a numpy array

In [19]:
z = np.array(z)

In [20]:
z

array([[[[0.3764706 , 0.5529412 , 0.2901961 ],
         [0.35686275, 0.5294118 , 0.2901961 ],
         [0.31764707, 0.49019608, 0.25882354],
         ...,
         [0.3764706 , 0.49019608, 0.40392157],
         [0.3137255 , 0.49019608, 0.32156864],
         [0.26666668, 0.49803922, 0.24705882]],

        [[0.38431373, 0.5529412 , 0.28235295],
         [0.5764706 , 0.7411765 , 0.48235294],
         [0.5764706 , 0.7372549 , 0.47058824],
         ...,
         [0.42745098, 0.54509807, 0.44313726],
         [0.6392157 , 0.8       , 0.6666667 ],
         [0.5686275 , 0.7764706 , 0.59607846]],

        [[0.26666668, 0.4392157 , 0.12941177],
         [0.24313726, 0.40784314, 0.10980392],
         [0.48235294, 0.63529414, 0.3254902 ],
         ...,
         [0.5686275 , 0.6901961 , 0.5568628 ],
         [0.72156864, 0.8745098 , 0.7607843 ],
         [0.63529414, 0.81960785, 0.69411767]],

        ...,

        [[0.11372549, 0.17254902, 0.08235294],
         [0.12941177, 0.1882353 , 0.09803922]

# Pridiction

In [21]:
res = mod.predict(z)

In [22]:
res = np.array(res)

# Getting the Submission File Ready

In [23]:
cl = train['breed'].unique()

In [25]:
c = res.argmax(axis = 1);
c.shape

(2361,)

In [26]:
ped = [cl [index] for index in c];
hehe = pd.DataFrame({'breed': ped})

In [27]:
hehe.set_index('breed').to_csv('submissionnew.csv')